In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split


2021-12-07 12:07:47.583879: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-07 12:07:47.583948: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv('genre.csv')

# прбираємо перші два стовпці 
df = df.iloc[:, 2:]

# вибираємо лише ті строки, 
# в яких мітки класів рівняються 5 і 10 
# що відповідають жанрам HipHop і Rock відповідно
df = df.query('Class == 5 | Class == 10').reset_index(drop=True)

# for col in df.columns:
#     pct_missing = np.mean(df[col].isnull())
#     print('{} - {}%'.format(col, round(pct_missing*100)))

df['Popularity'] = df['Popularity'].fillna(df['Popularity'].median())
df['key'] = df['key'].fillna(df['key'].median())
df['instrumentalness'] = df['instrumentalness'].fillna(df['instrumentalness'].median())

# змінюємо мітки класів
# HipHop: 0
# Rock: 1
df.loc[df['Class'] == 5, 'Class'] = 0
df.loc[df['Class'] == 10, 'Class'] = 1

X = df.drop(['Class'], axis = 1)
y = df['Class']

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

X_norm = scaler.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.33, random_state=42)

In [4]:
def build_encoder():
    model =Sequential()
    model.add(Dense(128, input_dim = 14, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy')
    return model

In [5]:
clf = KerasClassifier(build_encoder, epochs=30, batch_size=8)
clf.fit(x_train, y_train)

/tmp/ipykernel_2476/3814776099.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  clf = KerasClassifier(build_encoder, epochs=30, batch_size=8)
2021-12-07 12:07:50.358376: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-07 12:07:50.358549: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-07 12:07:50.358624: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-RJ3H6A8): /proc/driver/nvidia/version does not exist
2021-12-07 12:07:50.359059: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operati

Epoch 1/30
536/536 [==============================] - 2s 2ms/step - loss: 0.3123
Epoch 2/30
536/536 [==============================] - 1s 2ms/step - loss: 0.2220
Epoch 3/30
536/536 [==============================] - 1s 2ms/step - loss: 0.2067
Epoch 4/30
536/536 [==============================] - 1s 2ms/step - loss: 0.2023
Epoch 5/30
536/536 [==============================] - 1s 2ms/step - loss: 0.2010
Epoch 6/30
536/536 [==============================] - 1s 2ms/step - loss: 0.1936
Epoch 7/30
536/536 [==============================] - 1s 2ms/step - loss: 0.1937
Epoch 8/30
536/536 [==============================] - 1s 2ms/step - loss: 0.1929
Epoch 9/30
536/536 [==============================] - 1s 2ms/step - loss: 0.1843
Epoch 10/30
536/536 [==============================] - 1s 2ms/step - loss: 0.1898
Epoch 11/30
536/536 [==============================] - 1s 2ms/step - loss: 0.1887
Epoch 12/30
536/536 [==============================] - 1s 2ms/step - loss: 0.1837
Epoch 13/30
536/536 [====

In [6]:
y_tt = clf.predict(x_test)
1 - sum([abs(y1 - y2) for y1, y2 in zip(y_tt, y_test)]) / len(y_test)

array([0.93652297])